In [1]:
import yfinance as yf
import pandas as pd 
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet  import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

In [2]:
hj = datetime.today().strftime('%Y-%m-%d')
data_ini = '2010-07-13'
df_btc = yf.download('BTC-USD', data_ini, hj) 
df_btc.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-08-05,"$58,110.30","$58,268.83","$49,121.24","$53,991.46","$53,991.46",108991085584
2024-08-06,"$53,991.35","$57,059.92","$53,973.27","$56,034.32","$56,034.32",49300484106
2024-08-07,"$56,040.63","$57,726.88","$54,620.51","$55,027.46","$55,027.46",41637562185
2024-08-08,"$55,030.03","$62,673.77","$54,766.73","$61,710.14","$61,710.14",45298472567
2024-08-09,"$61,728.21","$61,751.86","$59,587.86","$60,880.11","$60,880.11",33425553115


In [3]:
df_btc.reset_index(inplace=True)
df_btc

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,$465.86,$468.17,$452.42,$457.33,$457.33,21056800
1,2014-09-18,$456.86,$456.86,$413.10,$424.44,$424.44,34483200
2,2014-09-19,$424.10,$427.83,$384.53,$394.80,$394.80,37919700
3,2014-09-20,$394.67,$423.30,$389.88,$408.90,$408.90,36863600
4,2014-09-21,$408.08,$412.43,$393.18,$398.82,$398.82,26580100
...,...,...,...,...,...,...,...
3610,2024-08-05,"$58,110.30","$58,268.83","$49,121.24","$53,991.46","$53,991.46",108991085584
3611,2024-08-06,"$53,991.35","$57,059.92","$53,973.27","$56,034.32","$56,034.32",49300484106
3612,2024-08-07,"$56,040.63","$57,726.88","$54,620.51","$55,027.46","$55,027.46",41637562185
3613,2024-08-08,"$55,030.03","$62,673.77","$54,766.73","$61,710.14","$61,710.14",45298472567


In [4]:
df = df_btc[["Date", "Adj Close"]]
df.rename(columns = {'Date' : 'ds', 'Adj Close' : 'y'}, inplace=True)

In [5]:
# Graph preço de fechamento
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.ds, y = df.y))

In [6]:
model = Prophet(seasonality_mode='additive')
model.fit(df)

22:12:29 - cmdstanpy - INFO - Chain [1] start processing
22:12:33 - cmdstanpy - INFO - Chain [1] done processing


In [7]:
# criar df com datas futuras
df_future = model.make_future_dataframe(periods=120)
df_future.tail()

,ds
3730,2024-12-03
3731,2024-12-04
3732,2024-12-05
3733,2024-12-06
3734,2024-12-07


In [8]:
previsao = model.predict(df_future)
previsao

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2014-09-17,$159.58,"$-8,269.69","$5,121.35",$159.58,$159.58,"$-1,994.23","$-1,994.23","$-1,994.23",$33.19,$33.19,$33.19,"$-2,027.42","$-2,027.42","$-2,027.42",$0.00,$0.00,$0.00,"$-1,834.65"
1,2014-09-18,$160.14,"$-8,435.78","$4,442.76",$160.14,$160.14,"$-2,084.93","$-2,084.93","$-2,084.93",$-16.25,$-16.25,$-16.25,"$-2,068.68","$-2,068.68","$-2,068.68",$0.00,$0.00,$0.00,"$-1,924.79"
2,2014-09-19,$160.71,"$-8,513.92","$4,294.55",$160.71,$160.71,"$-2,130.09","$-2,130.09","$-2,130.09",$-22.54,$-22.54,$-22.54,"$-2,107.56","$-2,107.56","$-2,107.56",$0.00,$0.00,$0.00,"$-1,969.38"
3,2014-09-20,$161.27,"$-8,482.88","$4,260.54",$161.27,$161.27,"$-2,159.98","$-2,159.98","$-2,159.98",$-16.20,$-16.20,$-16.20,"$-2,143.77","$-2,143.77","$-2,143.77",$0.00,$0.00,$0.00,"$-1,998.71"
4,2014-09-21,$161.84,"$-8,695.03","$3,935.53",$161.84,$161.84,"$-2,185.33","$-2,185.33","$-2,185.33",$-8.32,$-8.32,$-8.32,"$-2,177.01","$-2,177.01","$-2,177.01",$0.00,$0.00,$0.00,"$-2,023.50"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3730,2024-12-03,"$69,200.41","$61,023.61","$74,453.27","$67,216.71","$70,785.23","$-1,097.59","$-1,097.59","$-1,097.59",$-10.44,$-10.44,$-10.44,"$-1,087.15","$-1,087.15","$-1,087.15",$0.00,$0.00,$0.00,"$68,102.82"
3731,2024-12-04,"$69,267.03","$60,959.38","$75,310.01","$67,246.17","$70,865.46","$-1,068.26","$-1,068.26","$-1,068.26",$33.19,$33.19,$33.19,"$-1,101.45","$-1,101.45","$-1,101.45",$0.00,$0.00,$0.00,"$68,198.78"
3732,2024-12-05,"$69,333.65","$61,269.53","$75,097.77","$67,280.05","$70,979.22","$-1,131.99","$-1,131.99","$-1,131.99",$-16.25,$-16.25,$-16.25,"$-1,115.74","$-1,115.74","$-1,115.74",$0.00,$0.00,$0.00,"$68,201.66"
3733,2024-12-06,"$69,400.27","$60,870.05","$75,573.28","$67,323.56","$71,049.07","$-1,152.32","$-1,152.32","$-1,152.32",$-22.54,$-22.54,$-22.54,"$-1,129.78","$-1,129.78","$-1,129.78",$0.00,$0.00,$0.00,"$68,247.95"


In [9]:
previsao[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(120)

,ds,yhat,yhat_lower,yhat_upper
3615,2024-08-10,"$61,079.67","$54,569.71","$67,718.05"
3616,2024-08-11,"$61,157.54","$55,081.85","$67,781.40"
3617,2024-08-12,"$61,272.62","$54,772.03","$67,903.61"
3618,2024-08-13,"$61,283.90","$54,712.20","$67,726.53"
3619,2024-08-14,"$61,385.81","$54,860.28","$67,816.60"
...,...,...,...,...
3730,2024-12-03,"$68,102.82","$61,023.61","$74,453.27"
3731,2024-12-04,"$68,198.78","$60,959.38","$75,310.01"
3732,2024-12-05,"$68,201.66","$61,269.53","$75,097.77"
3733,2024-12-06,"$68,247.95","$60,870.05","$75,573.28"


In [10]:
# Graph 
plot_plotly(model, previsao)

In [11]:
plot_components_plotly(model, previsao)